In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/__results__.html
/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/__notebook__.ipynb
/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/__output__.json
/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/custom.css
/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/Gemma-medtr-2b-sft-v2/adapter_model.safetensors
/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/Gemma-medtr-2b-sft-v2/adapter_config.json
/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/Gemma-medtr-2b-sft-v2/README.md
/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/Gemma-medtr-2b-sft-v2/checkpoint-4499/adapter_model.safetensors
/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/Gemma-medtr-2b-sft-v2/checkpoint-4499/trainer_state.json
/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/Gemma-medtr-2b-sft-v2/checkpoint-4499/training_args.bin
/kaggle/input/finetune-gemma-with-peft

In [2]:
! pip install accelerate -q
! pip install -i https://pypi.org/simple/ bitsandbytes -q
! pip install peft -q
! pip install trl -q
! pip install --upgrade huggingface_hub -q
! pip install git+https://github.com/huggingface/datasets -U -q
! pip install git+https://github.com/huggingface/transformers -U -q

In [3]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
access_token_read = user_secrets.get_secret("HF_TOKEN")
login(token = access_token_read)

In [4]:
base_model_url = "google/gemma-2-2b"
new_model_url = "/kaggle/input/finetune-gemma-with-peft-4-bit-quantized-lora/Gemma-medtr-2b-sft-v2"

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format


# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model_url)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model_url,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
)

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [6]:
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
model = PeftModel.from_pretrained(base_model_reload, new_model_url)

model = model.merge_and_unload()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [7]:
model.save_pretrained("Gemma-medtr-2b-sft-v2")
tokenizer.save_pretrained("Gemma-medtr-2b-sft-v2")

('Gemma-medtr-2b-sft-v2/tokenizer_config.json',
 'Gemma-medtr-2b-sft-v2/special_tokens_map.json',
 'Gemma-medtr-2b-sft-v2/tokenizer.model',
 'Gemma-medtr-2b-sft-v2/added_tokens.json',
 'Gemma-medtr-2b-sft-v2/tokenizer.json')

In [8]:
model.push_to_hub("Gemma-medtr-2b-sft-v2", use_temp_dir=False)
tokenizer.push_to_hub("Gemma-medtr-2b-sft-v2", use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harishnair04/Gemma-medtr-2b-sft-v2/commit/8d11bfd8551c60a73e63525b8f4554bbf44169d8', commit_message='Upload tokenizer', commit_description='', oid='8d11bfd8551c60a73e63525b8f4554bbf44169d8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/harishnair04/Gemma-medtr-2b-sft-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='harishnair04/Gemma-medtr-2b-sft-v2'), pr_revision=None, pr_num=None)

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("harishnair04/Gemma-medtr-2b-sft-v1")
# model = AutoModelForCausalLM.from_pretrained("harishnair04/Gemma-medtr-2b-sft-v1")